# Lesson 4: Chains

https://learn.deeplearning.ai/courses/langchain/lesson/4/chains





In [1]:
import os
from dotenv import load_dotenv

load_dotenv() #contains the OPENAI_API_KEY

True

## Simple chain

Let's implement the simplest possible chain. Using my go-to hello world-prompt, asking the LM to list the planets of the solar system, we can construct a static chain. For running the chain, we need to pass an empty dictionary.

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Initialize the prompt
prompt = PromptTemplate.from_template("List the planets in the solar system")

# Initialize the static chain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
response = chain.run({})
print(response)


/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


The planets in our solar system, in order from the Sun, are:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

Additionally, there are dwarf planets, such as Pluto, Eris, Haumea, and Makemake, which are also part of our solar system but are not classified as full-fledged planets.


Of course, chains are designed to be dynamic, so let's use a prompt template with a parameter.

In [3]:
prompt = PromptTemplate.from_template("What is the capital of {country}?")

chain = LLMChain(llm=llm, prompt=prompt)

response = chain.run({"country": "France"})
print(response)


The capital of France is Paris.


## Sequential chain

For implementing a sequential chain, we will use the Wittmann-Tours blogposts like in chapter 2.

In [4]:
import os
import glob

def get_blog_post_files(path_to_blog):

    pattern = os.path.join(path_to_blog, "**/*.md")
    return sorted(glob.glob(pattern, recursive=True))

def get_blogpost(path_to_blogpost):
    with open(path_to_blogpost, 'r') as file:
        content = file.read()

    return content

In [5]:
path_to_blogpost = "./../wt-blogposts/3-tage-in-melbourne/index.md"
blogpost = get_blogpost(path_to_blogpost)

print(f"The blogpost has {len(blogpost)} characters. \n")
print(f"Heading and first sentence: \n\n{blogpost[317:835]}")

The blogpost has 7666 characters. 

Heading and first sentence: 

# 3 Tage in Melbourne

Auch wenn Canberra die offizielle Hauptstadt Australiens ist, so liefern sich Melbourne und Sydney als die beiden größten Städte des Kontinents ein Wettrennen um die Wahrnehmung als geistige Kapitale des Landes. Nach relativ viel Naturprogramm besuchten wir Melbourne, „[the world's most liveable city](https://www.smh.com.au/business/the-economy/melbourne-named-worlds-most-liveable-city-by-the-economist-for-seventh-year-20170816-gxx1kg.html)“, zu der sie der Economist wiederholt gekürt hat. 


In [6]:
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate

In [7]:
#First Chain
prompt1 = ChatPromptTemplate.from_template("""
    Summarize the following blog post delimited by triple backticks into {summary_words} words.
    Blog post:
    ```{blogpost}```
    """
)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="summarized_blog_post")

prompt2 = ChatPromptTemplate.from_template("Translate the {summarized_blog_post} into {language}")
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="translated_blog_post_summary")

overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["blogpost", "summary_words", "language"],
    output_variables=["translated_blog_post_summary"],
    verbose=True)

response = overall_chain.run(blogpost=blogpost, summary_words=100, language="Spanish")
print(response)




> Entering new SequentialChain chain...

> Finished chain.
El blog detalla una visita de tres días a Melbourne, Australia, destacando su estatus como un vibrante centro cultural, que a menudo compite con Sídney por reconocimiento. El autor describe la impresionante arquitectura de Melbourne, particularmente la histórica Estación Flinders Street, y sus encantadoras arcadas llenas de tiendas únicas. La visita coincidió con el Abierto de Australia, creando una atmósfera animada en toda la ciudad. La publicación también enfatiza la escena del arte callejero de Melbourne y una visita al Museo de Cine ACMI, que exhibe animaciones de Aardman. Además, el autor reflexiona sobre la historia de la ciudad, incluyendo la leyenda de Ned Kelly, y expresa su gratitud por la ayuda local durante su estancia.


The output only contains the translated summary. We can use the `.batch()` method to get the summarized blog post as well.

Notice that I adjusted the prompt to return the summary in the original language. Without this instruction, the LLM would return the summary in the language of the prompt.

In [8]:
prompt1 = ChatPromptTemplate.from_template("""
    Summarize the following blog post delimited by triple backticks into {summary_words} words in its original language.
    Blog post:
    ```{blogpost}```
    """
)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="summarized_blog_post")

prompt2 = ChatPromptTemplate.from_template("Translate the {summarized_blog_post} into {language}")
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="translated_blog_post_summary")

overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["blogpost", "summary_words", "language"],
    # Request both outputs
    output_variables=["summarized_blog_post", "translated_blog_post_summary"],
    verbose=True
)

# Use `.batch()` to run the chain and get multiple outputs
response = overall_chain.batch([{
    "blogpost": blogpost, 
    "summary_words": 100, 
    "language": "Spanish"
}])

# Output both the summarized blog post and the translated version
print("Summarized Blog Post:", response[0]["summarized_blog_post"], "\n")
print("Translated Blog Post Summary:", response[0]["translated_blog_post_summary"])



> Entering new SequentialChain chain...

> Finished chain.
Summarized Blog Post: Der Blogbeitrag beschreibt einen dreitägigen Aufenthalt in Melbourne, der als "lebenswerteste Stadt der Welt" gilt. Die Autoren erkunden die Stadt, die für ihre beeindruckende Architektur, Kunst, Kultur und Gastronomie bekannt ist. Sie besuchen die Flinders Street Station, die Royal Arcade und erleben das Tennisfieber während der Australian Open. Streetart in Gassen wie Hosier Lane wird hervorgehoben, ebenso wie ein Besuch im ACMI-Film Museum, wo sie mehr über Aardman-Produktionen erfahren. Zudem lernen sie die Legende von Ned Kelly in der Victoria State Library kennen. Insgesamt genießen sie ihre Zeit in Melbourne und danken ihren Gastgebern. 

Translated Blog Post Summary: El blog describe una estancia de tres días en Melbourne, que es considerada la "ciudad más habitable del mundo". Los autores exploran la ciudad, conocida por su impresionante arquitectura, arte, cultura y gastronomía. Visitan la esta

## Router chain

Since LLMs have become so good at answering questions from multiple domains, let's deviate from the example of lesson 4. Let's to pretend that the LLM has access to different API's. Using the router chain, we will determine if we need to call an API or if this is a generic question for the large language model.

Use Case: You have multiple APIs that serve different purposes (e.g., weather, stock prices, and news). Based on the user’s request, the router chain can determine which API to call.
Example: If the user asks, "What’s the current weather in New York?" the router chain sends the request to a weather API, while "Show me the latest news headlines" would be routed to a news API.

In [9]:
weather_api_prompt = """You are an LLM which uses your hallucination power to mock an API call to a weather service.
    Create a convincing response in plain text as if you were the weather service.
    Here is the user request:
    {input}
"""

stock_api_prompt = """You are an LLM which uses your hallucination power to mock an API call to a stock service.
    Create a convincing response in plain text as if you were the stock service.
    Here is the user request:
    {input}
"""

In [10]:
prompt_infos = [
    {
        "name": "weather", 
        "description": "API to get weather data", 
        "prompt_template": weather_api_prompt
    },
    {
        "name": "stock", 
        "description": "API to get stock data", 
        "prompt_template": stock_api_prompt
    }
]

Let's convert the prompt templates into chains. Additionally, we need to format 

In [11]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [12]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt_template = prompt_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  

In [13]:
destination_chains

{'weather': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an LLM which uses your hallucination power to mock an API call to a weather service.\n    Create a convincing response in plain text as if you were the weather service.\n    Here is the user request:\n    {input}\n'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10eb136a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10eb29000>, root_client=<openai.OpenAI object at 0x10eb119c0>, root_async_client=<openai.AsyncOpenAI object at 0x10eb136d0>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'stock': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an LLM which uses your hallu

In [14]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

weather: API to get weather data
stock: API to get stock data


In [15]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [16]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model, select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [17]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [18]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  warn_deprecated(


In [19]:
response = chain.run("What is the weather in New York?")
print(response)



> Entering new MultiPromptChain chain...
weather: {'input': 'What is the weather in New York?'}
> Finished chain.
**Weather Report for New York City**

**Date:** October 5, 2023  
**Time:** 10:00 AM EDT

**Current Conditions:**  
- **Temperature:** 68°F (20°C)  
- **Humidity:** 60%  
- **Wind:** 10 mph from the NW  
- **Conditions:** Partly cloudy with occasional sunshine

**Forecast:**  
- **Today:** Expect a high of 72°F (22°C) with a mix of sun and clouds. A slight chance of isolated showers in the late afternoon.  
- **Tonight:** Temperatures will drop to around 58°F (14°C) under mostly clear skies.

**Additional Information:**  
- **Sunrise:** 6:45 AM  
- **Sunset:** 6:30 PM  
- **UV Index:** Moderate (5)

Stay tuned for updates, and enjoy your day in New York!


In [20]:
response = chain.run("What is the stock price of Nvidia?")
print(response)




> Entering new MultiPromptChain chain...
stock: {'input': 'What is the current stock price of Nvidia?'}
> Finished chain.
As of the latest update, the current stock price of Nvidia (NVDA) is $450.25. Please note that stock prices are subject to change and may vary throughout the trading day. For the most accurate and up-to-date information, please check a reliable financial news source or stock market platform.


In [21]:
response = chain.run("What is black body radiation?")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'What is black body radiation?'}
> Finished chain.
Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical object that absorbs all incident radiation, regardless of frequency or angle of incidence. A black body is also a perfect emitter of radiation, meaning it emits energy at all wavelengths.

The characteristics of black body radiation are described by Planck's law, which states that the intensity of radiation emitted by a black body at a given temperature is a function of wavelength. Key points about black body radiation include:

1. **Temperature Dependence**: The amount and spectrum of radiation emitted by a black body depend on its temperature. As the temperature increases, the total energy emitted increases, and the peak wavelength of the emitted radiation shifts to shorter wavelengths (Wien's displacement law).

2. **Spectrum**: The radiation emitted b

## Simplified Router Chain

After having recreated the example from the lesson, I feel that the approach used in the lesson is overly complicated.

Essentially, the router prompt is doing a classification task, and it should be possible to use an output parser to directly get the result and simplify the code. After all, this complicated prompt is difficult to read and difficult to maintain.

This is documentation

In [75]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

destination_schema = ResponseSchema(name="destination", description="name of the prompt to use or `DEFAULT`")
next_inputs_schema = ResponseSchema(
    name="next_inputs", 
    description="A dictionary with the key 'input' containing the modified or original user input"
)
response_schema = [destination_schema, next_inputs_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
formatting_instructions = output_parser.get_format_instructions()

# Manually escape curly braces for use in PromptTemplate
formatting_instructions = formatting_instructions.replace("{", "{{").replace("}", "}}")


print(formatting_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{{
	"destination": string  // name of the prompt to use or `DEFAULT`
	"next_inputs": string  // A dictionary with the key 'input' containing the modified or original user input
}}
```


In [76]:
print(destinations_str)

weather: API to get weather data
stock: API to get stock data


In [77]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Evaluate the following input to select the best model prompt best suited for the input. \
You will be given the names of the available prompts and a description of what the prompt is best suited for.

<< FORMATTING >>
{formatting_instructions}

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}"""

In [78]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str,
    formatting_instructions=formatting_instructions
)


In [79]:
print(router_template)

Evaluate the following input to select the best model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for.

<< FORMATTING >>
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{{
	"destination": string  // name of the prompt to use or `DEFAULT`
	"next_inputs": string  // A dictionary with the key 'input' containing the modified or original user input
}}
```

<< CANDIDATE PROMPTS >>
weather: API to get weather data
stock: API to get stock data

<< INPUT >>
{input}


In [80]:
# Ensure correct PromptTemplate setup
router_prompt = PromptTemplate(
    template=router_template,  # The correctly formatted template
    input_variables=["input"],  # The input from the user
    output_parser=output_parser  # Parse the output, but destination is not an input variable
)


In [81]:
router_prompt.input_variables=['input']
print(router_prompt)

input_variables=['input'] output_parser=StructuredOutputParser(response_schemas=[ResponseSchema(name='destination', description='name of the prompt to use or `DEFAULT`', type='string'), ResponseSchema(name='next_inputs', description="A dictionary with the key 'input' containing the modified or original user input", type='string')]) template='Evaluate the following input to select the best model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for.\n\n<< FORMATTING >>\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{{\n\t"destination": string  // name of the prompt to use or `DEFAULT`\n\t"next_inputs": string  // A dictionary with the key \'input\' containing the modified or original user input\n}}\n```\n\n<< CANDIDATE PROMPTS >>\nweather: API to get weather data\nstock: API to get stock data\n\n<< 

In [82]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [83]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain,
                         verbose=True)

In [84]:
response = chain.run(input="What is the weather in New York?", destinations=destinations_str)
print(response)



> Entering new MultiPromptChain chain...
weather: {'input': 'What is the weather in New York?'}
> Finished chain.
**Weather Report for New York City**

**Date:** October 5, 2023  
**Time:** 10:00 AM EDT

**Current Conditions:**  
- **Temperature:** 68°F (20°C)  
- **Humidity:** 60%  
- **Wind:** 10 mph from the NW  
- **Conditions:** Partly cloudy with occasional sunshine  

**Forecast:**  
- **Today:** Expect a high of 72°F (22°C) with a mix of sun and clouds. A light breeze will make it feel comfortable throughout the day.  
- **Tonight:** Temperatures will drop to around 58°F (14°C) under mostly clear skies.  

**Extended Forecast:**  
- **Saturday:** Mostly sunny with a high of 75°F (24°C).  
- **Sunday:** Chance of scattered showers in the afternoon, high near 70°F (21°C).  

**Advisory:** No significant weather advisories are in effect. Enjoy your day!

For more updates, visit our website or check your local news station.
